In [2]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models as models
from torchvision import transforms
from torchvision import utils
from torchsummary import summary
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [3]:
torch.multiprocessing.set_start_method('spawn')

In [4]:
device = torch.device('cuda')

In [5]:
class EyeDataset(Dataset):
    def __init__(self, path, transform):
        self.transform = transform
        dir_names = os.listdir(path)
        dir_names.sort()
        self.label_list = []
        self.img_list = []
        self.file_names = []

        for dir_name in dir_names:
            if os.path.isdir(os.path.join(path, dir_name)):
                self.file_names = os.listdir(os.path.join(path, dir_name))
                self.file_names.sort()

                for file_name in self.file_names:
                    self.img_list.append(Image.fromarray(np.uint8(cv2.imread(os.path.join(path, dir_name, file_name)))))
                    self.label_list.append(int(file_name.split('_')[4]))

        self.img_transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img = self.img_list[idx]
        label = self.label_list[idx]

        if self.transform:
            img = self.img_transform(img)

        return img.to(device), label

In [10]:
dataset = EyeDataset(path='/mnt/data/dataset/mrlEyes_2018_01', transform=True)
train_size = int(len(dataset)*0.9)
val_size = (int(len(dataset)))-train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_data_loader = DataLoader(dataset=train_set, batch_size=10000, num_workers=0)
val_data_loader = DataLoader(dataset=val_set, batch_size=10000, num_workers=0)

In [11]:
model = models.resnet18(weights=None)

In [12]:
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)
summary(model, input_size=(3, 64, 64), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [14]:
epochs = 10

In [15]:
model.train()

for i in range(epochs):
    running_loss = 0

    with tqdm(train_data_loader, unit='batch', ncols=120) as tqdm_epochs:
        for train_data, train_label in tqdm_epochs:
            tqdm_epochs.set_description(f"Epoch {i+1}/{epochs}")
            outputs = model(train_data)
            # 오차 계산
            loss = loss_function(outputs, train_label.to(device))

            # 초기화
            optimizer.zero_grad()
            # 역전파
            loss.backward()
            # 스텝
            optimizer.step()

            predicted_classes = torch.max(outputs, 1)[1].cpu().numpy()
            train_label_cpu = train_label.cpu().numpy()

            precision = precision_score(train_label_cpu, predicted_classes, zero_division=0)
            recall = recall_score(train_label_cpu, predicted_classes, zero_division=0)
            f1 = f1_score(train_label_cpu, predicted_classes, zero_division=0)
            accuracy = accuracy_score(train_label_cpu, predicted_classes)

            tqdm_epochs.set_postfix(loss=f'{loss.item():.3f}', p=f'{precision:.2f}', r=f'{recall:.2f}', f1=f'{f1:.2f}', a=f'{accuracy:.2f}')

Epoch 10/10: 100%|████████████████████████| 8/8 [00:21<00:00,  2.68s/batch, a=0.99, f1=0.99, loss=0.020, p=1.00, r=0.99]


In [16]:
trans = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
img = cv2.imread('/mnt/data/dataset/mrlEyes_2018_01/s0001/s0001_03242_0_1_1_0_1_01.png')
img = Image.fromarray(np.uint8(img))
input_img = trans(img).unsqueeze(0)
input_img.shape

torch.Size([1, 3, 64, 64])

In [17]:
a = model(input_img.to(device))


In [18]:
model.eval()
with torch.no_grad():
    for val_data, val_label in val_data_loader:
        outputs = model(val_data)
        val_pred = torch.max(outputs, 1)[1].cpu().numpy()
        val_label_cpu = val_label.numpy()

        precision = precision_score(val_label_cpu, val_pred, zero_division=0)
        recall = recall_score(val_label_cpu, val_pred, zero_division=0)
        f1 = f1_score(val_label_cpu, val_pred, zero_division=0)
        accuracy = accuracy_score(val_label_cpu, val_pred)

        print(f'precision: {precision:.2f}, recall: {recall:.2f}, f1 score: {f1:.2f}, accuracy: {accuracy:.2f}')

precision: 0.97, recall: 0.99, f1 score: 0.98, accuracy: 0.98


In [19]:
torch.save(model.state_dict(), './eyes_model.pth')